In [1]:
# Requires at least Python 3.6, I believe
import sys
import pandas as pd
print(sys.version)
print(pd.__version__) # Must be at least 0.24.1 for IntegerArray

3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
0.25.2


In [2]:
# Display all columns - may take up a lot of memory
#pd.options.display.max_columns = None
#pd.options.display.float_format = '{:,.2f}'.format
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))   # A lot of memory?

In [3]:
# Hard-coded (but relative) path name. Note that ../data isn't populated it Git to save space
csl_df = pd.read_sas('../data/csl/csllinkedbypreg.sas7bdat', format = 'sas7bdat', encoding='iso-8859-1')
csl_df.head()

,Sitenum,MomID,Marital,Education,Insurance,Momage,Parity,Gravidity,Hxlivebirth,Hxmacrosomia,...,Fulldilattime,Placenttime,Inducttime,Ripentime,ROMtime,Epiduraltime,Intraantibtime,USdays,LMPdays,onepregflag
0,41,41-00001,1.0,3.0,1.0,34.0,4.0,5.0,4.0,1.0,...,15.0,4.0,361.0,NaN,300.0,343.0,346.0,103.0,271.0,2.0
1,41,41-00001,1.0,3.0,1.0,36.0,5.0,6.0,5.0,2.0,...,18.0,3.0,369.0,NaN,251.0,314.0,NaN,128.0,278.0,2.0
2,41,41-00002,1.0,3.0,1.0,37.0,1.0,3.0,1.0,1.0,...,NaN,1.0,NaN,NaN,1.0,4.0,NaN,128.0,268.0,1.0
3,41,41-00003,1.0,3.0,1.0,36.0,1.0,4.0,1.0,1.0,...,69.0,4.0,382.0,NaN,369.0,364.0,NaN,133.0,273.0,1.0
4,41,41-00004,1.0,2.0,1.0,34.0,2.0,6.0,2.0,1.0,...,12.0,5.0,339.0,NaN,105.0,116.0,455.0,204.0,274.0,1.0


In [4]:
%%time
# Version of convert_integer stored in util_mwb - Takes about 7-8 minutes
from util_mwb import convert_integer 
all_types = convert_integer(csl_df)
csl_df.head()  # Note - the util_mwb.convert_integer function changes the DataFrame *in place*
#CPU times: user 5min 22s, sys: 4min 31s, total: 9min 53s
#Wall time: 7min 34s

CPU times: user 5min 22s, sys: 4min 31s, total: 9min 53s
Wall time: 7min 34s


,Sitenum,MomID,Marital,Education,Insurance,Momage,Parity,Gravidity,Hxlivebirth,Hxmacrosomia,...,Fulldilattime,Placenttime,Inducttime,Ripentime,ROMtime,Epiduraltime,Intraantibtime,USdays,LMPdays,onepregflag
0,41,41-00001,1,3,1,34,4,5,4,1,...,15,4,361,NaN,300,343,346,103,271,2
1,41,41-00001,1,3,1,36,5,6,5,2,...,18,3,369,NaN,251,314,NaN,128,278,2
2,41,41-00002,1,3,1,37,1,3,1,1,...,NaN,1,NaN,NaN,1,4,NaN,128,268,1
3,41,41-00003,1,3,1,36,1,4,1,1,...,69,4,382,NaN,369,364,NaN,133,273,1
4,41,41-00004,1,2,1,34,2,6,2,1,...,12,5,339,NaN,105,116,455,204,274,1


In [5]:
# Save off the types dictionary as a pickle file for later use
import pickle
#with open('csl_types.pickle', 'wb') as file:
#    pickle.dump(all_types, file)  # Commented out to avoid overwrite
    
with open('csl_types.pickle', 'rb') as file:
    read_types = pickle.load(file)
    
print( all_types == read_types)
# True

True


In [6]:
# Original code (copied to util_mwb); left here if optimization needed

# Convert any non-float fields to IntegerArray (Int)
# Note than IntegerArrays are an experimental addition in Pandas 0.24. They
# allow integer columns to contain NaN fields like float columns.
#
# This is a rather brute-force technique that loops through every column
# and every row. There's got to be a more efficient way to do it since it 
# takes a long time and uses up a lot of memory.
def convert_integer (df):
    type_dict = {}
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':   # Assuming dtype is "float64"
            # TODO: Need to remove inner loop - SLOW!
            for val in df[col]:
                # If not NaN and the int() value is different from
                # the float value, then we have an actual float.
                if pd.notnull(val) and abs(val - int(val)) > 1e-6:
                    intcol_flag = False
                    break;
            # If not a float, change it to an Int based on size
            if intcol_flag:
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
#        print(f"{col} is {df[col].dtype}")
        type_dict[col] = df[col].dtype
    return df

In [11]:
%%time
df = convert_integer(csl_df)
# CPU times: user 3min, sys: 2min 14s, total: 5min 14s
# Wall time: 5min 14s

# Much, much faster second time
#CPU times: user 1.54 s, sys: 1.16 ms, total: 1.54 s
#Wall time: 1.54 s

CPU times: user 1.54 s, sys: 1.16 ms, total: 1.54 s
Wall time: 1.54 s


In [8]:
# Attempt to improve speed, but there was no improvement

# Convert any non-float fields to IntegerArray (Int)
# Note than IntegerArrays are an experimental addition in Pandas 0.24. They
# allow integer columns to contain NaN fields like float columns.
#
# This is a rather brute-force technique that loops through every column
# and every row. There's got to be a more efficient way to do it since it 
# takes a long time and uses up a lot of memory.
def convert_to_integer (df):
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':   # Assuming dtype is "float64"
            # If not NaN and the int() value is different from
            # the float value, then we have an actual float.
            # s = df[col].apply(lambda x: pd.notnull(x) and abs(x - int(x)) > 1e-6)
            s = df[col].notnull() & df[col].sub(df[col].round()).abs().gt(1e-6)
            if s.any():
                intcol_flag = False
            # If not a float, change it to an Int based on size
            if intcol_flag:
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
#        print(f"{col} is {df[col].dtype}")
    return df

In [9]:
%%time
df = convert_to_integer(csl_df)

CPU times: user 1.52 s, sys: 99.7 ms, total: 1.62 s
Wall time: 414 ms


In [12]:
types = csl_df.dtypes
print(type(types))
types.value_counts()

<class 'pandas.core.series.Series'>


Int8       663
float64     59
Int16       38
object      17
Int32        2
dtype: int64

In [13]:
import numpy as np
np.random.seed(10)
df = pd.DataFrame(np.random.choice([1, 2, 3.3, 5000, 111111, np.NaN], (3,9)), 
                  columns=[f'col{i}' for i in range(9)])
df

,col0,col1,col2,col3,col4,col5,col6,col7,col8
0,2.0,NaN,111111.0,1.0,2.0,5000.0,111111.0,2.0,NaN
1,1.0,NaN,2.0,3.3,1.0,2.0,1.0,3.3,1.0
2,111111.0,5000.0,1.0,111111.0,5000.0,1.0,5000.0,3.3,2.0


In [14]:
import util_mwb
util_mwb.convert_integer(df)

{'col0': Int32Dtype(),
 'col1': Int16Dtype(),
 'col2': Int32Dtype(),
 'col3': dtype('float64'),
 'col4': Int16Dtype(),
 'col5': Int16Dtype(),
 'col6': Int32Dtype(),
 'col7': dtype('float64'),
 'col8': Int8Dtype()}

In [18]:
s = pd.cut(df.max(), bins=[0, 127, 32767, 2147483647], labels=['Int8', 'Int16', 'Int32'])
print(s)
s = s.where(df.dtypes=='float') # So we only cast the previously float cols 
print(s)
            
#for idx, gp in s.groupby(s):
#    df.loc[:, gp.index] = df.loc[:, gp.index].astype(idx)            

col0    Int32
col1    Int16
col2    Int32
col3    Int32
col4    Int16
col5    Int16
col6    Int32
col7     Int8
col8     Int8
dtype: category
Categories (3, object): [Int8 < Int16 < Int32]
